In [1]:
import pandas as pd
import tensorflow as tf
import sklearn

In [3]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [6]:
base = pd.read_csv('games.csv')
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1)

In [9]:
base.shape

(16719, 13)

In [10]:
base.isnull().sum()

,0
Name,2
Platform,0
Year_of_Release,269
Genre,2
Publisher,54
NA_Sales,0
EU_Sales,0
JP_Sales,0
Critic_Score,8582
Critic_Count,8582


In [11]:
base = base.dropna(axis = 0)

In [12]:
base.shape

(6825, 13)

In [13]:
base.isnull().sum()

,0
Name,0
Platform,0
Year_of_Release,0
Genre,0
Publisher,0
NA_Sales,0
EU_Sales,0
JP_Sales,0
Critic_Score,0
Critic_Count,0


In [16]:
base['Name'].value_counts()

,count
Name,
Madden NFL 07,8
Need for Speed: Most Wanted,8
LEGO Star Wars II: The Original Trilogy,8
Terraria,7
Madden NFL 08,7
...,...
Brain Age: Train Your Brain in Minutes a Day,1
Wii Fit,1
Wii Sports Resort,1


In [17]:
base = base.drop('Name', axis = 1)

In [18]:
base.shape

(6825, 12)

In [19]:
base.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating'],
      dtype='object')

In [20]:
X = base.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values
X

array([['Wii', 2006.0, 'Sports', ..., '8', 322.0, 'E'],
       ['Wii', 2008.0, 'Racing', ..., '8.3', 709.0, 'E'],
       ['Wii', 2009.0, 'Sports', ..., '8', 192.0, 'E'],
       ...,
       ['PC', 2014.0, 'Action', ..., '7.6', 412.0, 'M'],
       ['PC', 2011.0, 'Shooter', ..., '5.8', 43.0, 'T'],
       ['PC', 2011.0, 'Strategy', ..., '7.2', 13.0, 'E10+']], dtype=object)

In [23]:
y_na = base.iloc[:, 4].values
y_eu = base.iloc[:, 5].values
y_jp = base.iloc[:, 6].values

In [24]:
y_na, y_eu, y_jp

(array([4.136e+01, 1.568e+01, 1.561e+01, ..., 0.000e+00, 1.000e-02,
        0.000e+00]),
 array([2.896e+01, 1.276e+01, 1.093e+01, ..., 1.000e-02, 0.000e+00,
        1.000e-02]),
 array([3.77, 3.79, 3.28, ..., 0.  , 0.  , 0.  ]))

In [25]:
base ['Platform'].value_counts()

,count
Platform,
PS2,1140
X360,858
PS3,769
PC,651
XB,565
Wii,479
DS,464
PSP,390
GC,348


In [26]:
onehorencoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0, 2, 3, 8])], remainder='passthrough')
X = onehorencoder.fit_transform(X).toarray()

In [28]:
X.shape

(6825, 303)

In [30]:
X[0]

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 

In [31]:
camada_entrada = Input(shape = (303,))
camada_oculta1 = Dense(units = 153, activation = 'relu')(camada_entrada)
camada_oculta2 = Dense(units = 153, activation = 'relu')(camada_oculta1)
camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida2 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation = 'linear')(camada_oculta2)

In [32]:
regressor = Model(inputs = camada_entrada, outputs = [camada_saida1, camada_saida2, camada_saida3])

In [33]:
regressor.compile(optimizer = 'adam', loss = 'mse')

In [34]:
regressor.fit(X, [y_na, y_eu, y_jp], epochs = 500, batch_size = 100)

Epoch 1/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - dense_2_loss: 3920.5474 - dense_3_loss: 4074.7949 - dense_4_loss: 163.8149 - loss: 8159.9580
Epoch 2/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - dense_2_loss: 4.5734 - dense_3_loss: 4.7109 - dense_4_loss: 0.9938 - loss: 10.2792
Epoch 3/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - dense_2_loss: 1.3706 - dense_3_loss: 1.4801 - dense_4_loss: 0.4004 - loss: 3.2516
Epoch 4/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - dense_2_loss: 0.9804 - dense_3_loss: 1.2699 - dense_4_loss: 0.2840 - loss: 2.5347
Epoch 5/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dense_2_loss: 1.8183 - dense_3_loss: 1.3866 - dense_4_loss: 0.2902 - loss: 3.4955
Epoch 6/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - dense_2_loss: 1.2383 - dense_3_loss: 1.0375 - dense_4_loss: 0.2273 - loss: 2.5035
Epoch 7/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - dense_2_loss: 0.9662 - dense_3_loss: 1.1648 - dense_4_loss: 0.2989 - loss: 2.4305
Epoch 8/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 6

In [35]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(X)

214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [36]:
previsao_na, previsao_na.mean()

(array([[0.42598996],
        [0.8634006 ],
        [0.35367802],
        ...,
        [0.40200892],
        [0.2574799 ],
        [0.24036641]], dtype=float32),
 np.float32(0.33814153))

In [37]:
y_na, y_na.mean()

(array([4.136e+01, 1.568e+01, 1.561e+01, ..., 0.000e+00, 1.000e-02,
        0.000e+00]),
 np.float64(0.3944835164835165))

In [38]:
from sklearn.metrics import mean_absolute_error

In [39]:
mean_absolute_error(y_na, previsao_na)

0.3510793337632448

In [40]:
previsao_eu, previsao_eu.mean()

(array([[0.34495023],
        [0.6326614 ],
        [0.22041081],
        ...,
        [0.3181383 ],
        [0.12003949],
        [0.10218358]], dtype=float32),
 np.float32(0.19080833))

In [41]:
y_eu, y_eu.mean()

(array([2.896e+01, 1.276e+01, 1.093e+01, ..., 1.000e-02, 0.000e+00,
        1.000e-02]),
 np.float64(0.23608937728937732))

In [42]:
mean_absolute_error(y_eu, previsao_eu)

0.21669451360047517

In [43]:
previsao_jp, previsao_jp.mean()

(array([[0.1019517 ],
        [0.1795428 ],
        [0.12325734],
        ...,
        [0.08523823],
        [0.04442482],
        [0.03040062]], dtype=float32),
 np.float32(0.06849871))

In [44]:
y_jp, y_jp.mean()

(array([3.77, 3.79, 3.28, ..., 0.  , 0.  , 0.  ]),
 np.float64(0.06415824175824175))

In [45]:
mean_absolute_error(y_jp, previsao_jp)

0.09872314356604775